In [1]:
import torch
from chunkformer_vpb.training.finetune_utils import (
    get_default_args,
    prepare_input_file,
    load_model_only,
    compute_chunkformer_loss,
)
from chunkformer_vpb.training.tokenizer import normalize_vi, GreedyTokenizer
from torch.nn.utils.rnn import pad_sequence

import os, math, torch
from chunkformer_vpb.training.finetune_config import FinetuneConfig
from chunkformer_vpb.training.data_loader     import get_dataloaders
from chunkformer_vpb.training.optimizer       import build_model_and_optimizer
from chunkformer_vpb.training.finetune_utils  import compute_loss_batch_v1, compute_loss_batch_v2

from chunkformer_vpb.training.finetune_config import FinetuneConfig
from chunkformer_vpb.training.data_loader     import get_dataloaders

def debug_text_pipeline(label_text: str, tokenizer: GreedyTokenizer):
    print(">>> Text pipeline debug <<<")
    print("  Original text :", label_text)
    norm = normalize_vi(label_text)
    print("  Normalized   :", norm)
    ids = tokenizer.tokenize(norm)
    print("  Token IDs    :", ids)
    try:
        dec = tokenizer.decode_ids(ids)
    except AttributeError:
        # nếu chưa có decode_ids
        dec = "".join(tokenizer.vocab[id] for id in ids)
    print("  Decoded      :", dec)
    print()

def main():
    CFG_PATH = "../../config/finetune_config.yaml"
    DEBUG_STEPS = 5         # số batch muốn test
    DEVICE      = "cpu" # "cuda" if torch.cuda.is_available() else "cpu"

    cfg = FinetuneConfig.from_yaml(CFG_PATH)

    # để nhanh: batch nhỏ & shuffle false
    cfg.training.batch_size = 1
    cfg.training.shuffle    = False

    train_loader, _ = get_dataloaders(cfg)

    model, _, optim, sched = build_model_and_optimizer(
        cfg, torch.device(DEVICE), total_steps=DEBUG_STEPS
    )

    count = 0
    for step, (feats, feat_lens, toks, tok_lens) in enumerate(train_loader, 1):
        print("\n=== DataLoader batch debug ===")
        print(f"Step: {step}")
        print("loader_feats.shape    :", feats.shape)
        print("loader_feat_lens      :", feat_lens)
        print("loader_toks.shape     :", toks.shape)
        print("loader_tok_lens       :", tok_lens)
        
        # Nếu bạn đã có feats, feat_lens, toks, tok_lens tự chuẩn bị:
        # In ra shape, giá trị, hoặc compare từng phần tử
        
        print(">>> Batch‐of‐1 loss <<<")
        loss_b, loss_b_ctc, loss_b_att = compute_loss_batch_v1(
            model, feats, feat_lens, toks, tok_lens, cfg, DEVICE
        )
        print(f"Batch1: loss={loss_b:.3f},  ctc={loss_b_ctc:.3f}, att={loss_b_att:.3f}")

        
        # Chỉ chạy 1 batch để debug
        count += 1
        print("\n\n=========================================\n\n")
        if count >= DEBUG_STEPS:
            print(">>> Debug complete <<<")
            break

if __name__ == "__main__":
    main()


/home/kylh/.local/share/mamba/envs/stt310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-07-17 15:51:57] INFO: Checkpoint: loading from checkpoint ../../../chunkformer-large-vie/pytorch_model.bin for GPU



🧾 Loaded checkpoint from: ../../../chunkformer-large-vie/pytorch_model.bin
📦 Checkpoint keys: ['encoder.global_cmvn.mean', 'encoder.global_cmvn.istd', 'encoder.embed.out.weight', 'encoder.embed.out.bias', 'encoder.embed.conv.0.weight'] ... (total 813)
🔍 AED decoder head included in checkpoint? ✅ YES
📊 Model total params: 113,852,240, trainable: 113,852,240
!!!Vocab size: 6992

=== DataLoader batch debug ===
Step: 1
loader_feats.shape    : torch.Size([1, 626, 80])
loader_feat_lens      : tensor([626])
loader_toks.shape     : torch.Size([1, 15])
loader_tok_lens       : tensor([15])
>>> Batch‐of‐1 loss <<<
[DBG] loss_ctc = 0.1989
>>>>>>>>>>>>>>>>>>>  ys_out shape: torch.Size([1, 16]), logp shape: torch.Size([1, 16, 6992])
[DBG] loss_att = 0.7648
[DBG] final loss = 0.5950
Batch1: loss=0.595,  ctc=0.199, att=0.765





=== DataLoader batch debug ===
Step: 2
loader_feats.shape    : torch.Size([1, 573, 80])
loader_feat_lens      : tensor([573])
loader_toks.shape     : torch.Size([1, 15])
loa